### Operational Profile

This file allows the user to generate a visualisation charge/discharge profile of BESS & state of charge of BESS.

1. Inputs
raw_output_filename:str - the model output filename that the user wants to analyse
random_day:pd.datetime - the date the user desires to view

2. Outputs
figure:plotlyfig - a plotly figure

In [99]:
import os 
import pandas as pd
import plotly.express as px
import nbformat
import plotly.graph_objects as go

# set some default colors
zenobe_grey = "#9B9A9A"
zenobe_red = "#F37676"
zenobe_black = "#000000"
zenobe_blue = "#4d76ff"
zenobe_green ="#008000"

graph_primary = "rgba(230,98,5,1)" 
graph_primary_med = "rgba(230,98,5,0.75)" 
graph_primary_light = "rgba(230,98,5,0.4)"
graph_primary_v_light = "rgba(230,98,5,0.1)"
graph_secondary = "rgb(144,144,144)"
graph_secondary_med = "rgba(144,144,144,0.75)" 
graph_secondary_light = "rgba(144,144,144,0.5)"
graph_secondary_v_light = "rgba(144,144,144,0.2)"


font_size = 14
font_family = "Century Gothic"
num_time_periods = 48
font_color = "black"
project_name = "Example"
logo_filename = "zenobe_logo.png"
project_power_capacity = 100
project_energy_capacity = 100



In [100]:
# Read in data
# Get the current directory of the script
current_directory = os.path.dirname(os.path.abspath(os.getcwd()))

# Get the parent directory
parent_directory = os.path.dirname(current_directory)
raw_output_filename = "example_02-07.csv"
optimiser_output_df = pd.read_csv(os.path.join(parent_directory,"optimisation","data_output","raw_output",raw_output_filename),index_col=[0])
optimiser_output_df.index = pd.to_datetime(optimiser_output_df.index)


In [101]:
# Choose a day to analyse 
random_period_start= pd.to_datetime("2023-01-20")
random_period_end= pd.to_datetime("2023-01-23")
random_period = optimiser_output_df.loc[(optimiser_output_df.index>=random_period_start)&(optimiser_output_df.index<=random_period_end)]


In [102]:

from plotly.subplots import make_subplots
fig = make_subplots(rows=3,cols=1,specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]])


In [103]:

# Import Export and BM Actions
trace1 = go.Bar(x=random_period.index,
                y=-random_period["import_da_vol"],
                name='Day Ahead',
                marker_color=graph_primary,
                yaxis='y1'
                    )
trace2 = go.Bar(x=random_period.index,
                y=random_period["export_da_vol"],
                name='',
                marker_color=graph_primary,
                yaxis='y1',
                showlegend=False
                    )
trace3 = go.Scatter(x=random_period.index,
                        y=random_period["day_ahead_price"],
                        name='Day Ahead Price',
                        mode='lines',
                        marker_color=graph_primary,
                        line=dict(width=2, 
                        color=graph_primary, 
                        dash='dash'),
                        yaxis='y2')
data = [trace1,trace2]

fig.update_layout(barmode='stack')

fig.update_yaxes(title_text="Power (MW)",row=1,col=1)   
fig.update_yaxes(title_text= "Price (£/MWh)",row=1,col=1,secondary_y=True) 
for trace in data:
    fig.add_trace(trace,row=1,col=1)
fig.add_trace(trace3,row=1,col=1,secondary_y=True)

In [104]:


trace1 = go.Bar(x=random_period.index,
                y=-random_period["import_intraday_vol"],
                name="Intraday",
                marker_color=graph_secondary,
                yaxis='y1'
                    )


trace2 = go.Bar(x=random_period.index,
                y=random_period["export_intraday_vol"],
                name='Export Intraday',
                marker_color=graph_secondary,
                yaxis='y1',
                showlegend=False
                    )
trace3 = go.Scatter(x=random_period.index,
                        y=random_period["intraday_price"],
                        name='Intraday Price',
                        mode='lines',
                        marker_color=graph_secondary,
                        line=dict(width=2, 
                        color=graph_secondary, 
                        dash='dash'),
                        yaxis='y2')
data = [trace1,trace2]

fig.update_layout(barmode='stack')

fig.update_yaxes(title_text="Power (MW)",row=2,col=1)   
fig.update_yaxes(title_text= "Price (£/MWh)",row=2,col=1,secondary_y=True) 
for trace in data:
    fig.add_trace(trace,row=2,col=1)
fig.add_trace(trace3,row=2,col=1,secondary_y=True)


In [105]:
#SoC
trace1 = go.Scatter(x=random_period.index,
                        y=(random_period["soc_intraday"]/project_energy_capacity)*100,
                        name='SoC (%)',
                        mode='lines',
                        marker_color=graph_secondary,
                        line=dict(width=2, 
                        color='black', 
                        dash='dash'),
                        yaxis='y2')

soc_data = [trace1]


fig.update_yaxes(title_text= "SoC (%)",range=[0,110], row=3,col=1,secondary_y=False) 


for trace in soc_data:
    fig.add_trace(trace,row=3,col=1)

In [106]:
fig.update_layout(barmode='overlay',
                  font=dict(
        family=font_family,
        size=font_size,
        color= zenobe_black,
    ),)
# Update the layout to show time on x-axis

fig.update_layout(title=dict(x = 0.5,
                            xanchor =  'center',
                            yanchor = 'top',text=f"Example Operation from {random_period_start.date()} to {random_period_end.date()}<br><sup>{project_power_capacity}MW/{project_energy_capacity}MWh BESS, 2 cycles max"),
                    
)
fig.update_layout(
    margin=dict(b=10),
    annotations=[
        go.layout.Annotation(
            x=0,  # X position on the axis (0 is the far left, 1 is far right)
            y=-0.2,  # Y position below the plot (negative value to place it below)
            xref='paper',  # 'paper' makes the annotation relative to the entire plot (not data coordinates)
            yref='paper',
            text=f"Notes: Assumes perfect forecast at Day Ahead, with cycling reserved for intraday cycling",  # The note text
            showarrow=False,  # No arrow
            xanchor='left',  # Align the text to the left
            yanchor='top',  # Anchor the note to the top of the y-position
            font=dict(size=14)  # Customize the font size if needed
        ),
        
    ]
)
fig.add_layout_image(
    dict(
        source="images/zenobe_logo.PNG",  # Path to the logo
        xref="paper", yref="paper",
        x=1.2, y=1.05,  # Positioning the logo at the top right
        sizex=.25, sizey=.25,  # Adjust size based on your needs
        xanchor="right", yanchor="bottom"
    )
)
fig.update_layout(
    paper_bgcolor='rgb(256,256,256)',
    plot_bgcolor='rgb(256,256,256)',
    font_color="black",
    barmode="relative",
    width=1000,
    height=600,
)
fig.show()